In [2]:
import os
import sys
import time
import glob
import json
import torch
import torch.nn as nn
import yaml
import numpy as np
import pickle as pkl
import os.path as op
import pytorch_lightning as pl
import matplotlib.pyplot as plt
from pathlib2 import Path

sys.path.append('..')
from scripts.model.unet_interface import ModelInteface

%load_ext autoreload
%autoreload 2

In [3]:
def time_string():
    """ Generate a time string from year to second.
    """
    return time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())

def get_model_path(date, root='/gypsum/scratch1/zhongyangzha/DVS_HPE/dvs-hpe-light/lightning_logs', best=True):
    model_subdir = op.join(root, date, 'checkpoints')
    if best:
        model_path = glob.glob(op.join(model_subdir, 'best*'))[0]
    else:
        model_path = op.join(model_subdir, 'last.ckpt')
    return model_path

def load_model(time_str='05-25-16-19-04', best=True, device='cuda'):
    model_path = get_model_path(time_str, best=True)
    model = ModelInteface.load_from_checkpoint(model_path)
    model = model.to(device)
    print('Using trained model: ', model_path)
    return model

In [18]:
root = '/gypsum/scratch1/zhongyangzha/DVS_HPE/U2NetTORE/lightning_logs/'
model_ori_path = get_model_path('06-10-21-14-08', root=root, best=True) #05-19-18-28-41
model_new_path = model_ori_path.replace('.ckpt', '.pt')

model_ori = ModelInteface.load_from_checkpoint(model_ori_path)
torch.save(model_ori.model, model_new_path)

Model hparams saved!
Output channel number: 16
Model hparams saved!
Output channel number: 16


In [11]:
model_new_path = './results/model.pt'
torch.save(model_ori.model, model_new_path)

In [12]:
model_loaded = torch.load(model_new_path)

In [13]:
model_loaded.eval()

StudentUnetAdv(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(6, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=Fa

In [14]:
def predict_mask(unet, x):
    x1 = unet.inc(x)
    x2 = unet.down1(x1)
    x3 = unet.down2(x2)
    x4 = unet.down3(x3)
    x5 = unet.down4(x4)

    x = unet.up1(x5, x4)
    x = unet.up2(x, x3)
    x = unet.up3(x, x2)
    x = unet.up4(x, x1)

    masks = unet.outc(x).squeeze()
    masks = torch.sigmoid(masks)[:,0:1]
    return masks

In [16]:
x=torch.rand(1,6,260,346)
predict_mask(model_loaded, x).shape

torch.Size([16, 1, 346])